In [4]:
# Setup Dash with authentication
from dash import Dash, callback_context

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html, dash_table
import plotly.express as px
import plotly.graph_objects as go
from dash.dependencies import Input, Output, State
import base64
import json
from datetime import datetime

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd

# Import the enhanced CRUD module with authentication and analytics
from Artifact_Three_animal_shelter_CRUD import AnimalShelter

###########################
# Data Manipulation / Model
###########################

# Connect to MongoDB via CRUD Module with error handling
print("Initializing Animal Shelter Database Connection...")
try:
    db = AnimalShelter()
    print("Database connection successful!")
except Exception as e:
    print(f"Database connection error: {e}")
    print("Creating mock data for demonstration...")
    # Create mock data structure for demonstration
    df = pd.DataFrame({
        'animal_id': ['A1', 'A2', 'A3', 'A4', 'A5'],
        'animal_type': ['Dog', 'Cat', 'Dog', 'Dog', 'Cat'],
        'breed': ['Labrador Retriever Mix', 'Siamese', 'German Shepherd', 'Golden Retriever', 'Persian'],
        'age_upon_outcome_in_weeks': [52, 26, 78, 45, 32],
        'outcome_type': ['Adoption', 'Transfer', 'Adoption', 'Return to Owner', 'Adoption'],
        'sex_upon_outcome': ['Intact Female', 'Spayed Female', 'Intact Male', 'Neutered Male', 'Spayed Female'],
        'name': ['Buddy', 'Whiskers', 'Max', 'Charlie', 'Luna']
    })
    # Create a mock db object for fallback
    db = None
else:
    # Get initial data 
    print("Loading initial data...")
    try:
        # Use the new method that doesn't require authentication
        df = pd.DataFrame.from_records(db.read_without_auth({}))
        print(f"Successfully loaded {len(df)} records")
    except Exception as e:
        print(f"Error loading data: {e}")
        # Create mock dataframe as fallback
        df = pd.DataFrame({
            'animal_id': ['A1', 'A2', 'A3', 'A4', 'A5'],
            'animal_type': ['Dog', 'Cat', 'Dog', 'Dog', 'Cat'],
            'breed': ['Labrador Retriever Mix', 'Siamese', 'German Shepherd', 'Golden Retriever', 'Persian'],
            'age_upon_outcome_in_weeks': [52, 26, 78, 45, 32],
            'outcome_type': ['Adoption', 'Transfer', 'Adoption', 'Return to Owner', 'Adoption'],
            'sex_upon_outcome': ['Intact Female', 'Spayed Female', 'Intact Male', 'Neutered Male', 'Spayed Female'],
            'name': ['Buddy', 'Whiskers', 'Max', 'Charlie', 'Luna']
        })

# Authentication state
current_user = None
auth_token = None
user_role = None

# Define img_html variable
img_html = html.Img(src="", style={'width': '200px', 'height': '150px'})

#########################
# Dashboard Layout / View
#########################
app = Dash(__name__)

# Make db accessible to callbacks by storing it in app
app.db = db
app.df = df

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Austin Animal Shelter Dashboard'))),
    html.Center("Ericka Resendez Dashboard With Authentication & Advanced Analytics"),
    
    # Authentication Section 
    html.Div([
        html.H4("User Authentication", style={'color': '#2c3e50', 'textAlign': 'center'}),
        html.Div([
            dcc.Input(id='username-input', type='text', placeholder='Username', 
                     style={'margin': '5px', 'padding': '8px'}),
            dcc.Input(id='password-input', type='password', placeholder='Password',
                     style={'margin': '5px', 'padding': '8px'}),
            html.Button('Login', id='login-btn', n_clicks=0,
                       style={'background-color': '#3498db', 'color': 'white', 'border': 'none',
                              'padding': '8px 15px', 'border-radius': '5px', 'cursor': 'pointer', 'margin': '5px'}),
            html.Button('Logout', id='logout-btn', n_clicks=0,
                       style={'background-color': '#e74c3c', 'color': 'white', 'border': 'none',
                              'padding': '8px 15px', 'border-radius': '5px', 'cursor': 'pointer', 'margin': '5px', 'display': 'none'}),
            html.Div(id='auth-status', style={'marginTop': '10px'}),
            html.Div(id='user-info', style={'marginTop': '10px', 'fontWeight': 'bold'})
        ], style={'textAlign': 'center', 'padding': '15px', 'border': '1px solid #bdc3c7', 'border-radius': '5px'})
    ], id='auth-section'),
    
    # User Management Section Admin only
    html.Div([
        html.H4("User Management", style={'color': '#2c3e50', 'textAlign': 'center'}),
        html.Div([
            dcc.Input(id='new-username', type='text', placeholder='New Username', 
                     style={'margin': '5px', 'padding': '8px'}),
            dcc.Input(id='new-password', type='password', placeholder='New Password',
                     style={'margin': '5px', 'padding': '8px'}),
            dcc.Input(id='new-email', type='email', placeholder='Email',
                     style={'margin': '5px', 'padding': '8px'}),
            dcc.Dropdown(
                id='new-role',
                options=[
                    {'label': 'Viewer', 'value': 'viewer'},
                    {'label': 'Analyst', 'value': 'analyst'},
                    {'label': 'Admin', 'value': 'admin'}
                ],
                placeholder='Select Role',
                style={'margin': '5px', 'width': '200px'}
            ),
            html.Button('Create User', id='create-user-btn', n_clicks=0,
                       style={'background-color': '#27ae60', 'color': 'white', 'border': 'none',
                              'padding': '8px 15px', 'border-radius': '5px', 'cursor': 'pointer', 'margin': '5px'}),
            html.Button('Refresh Users', id='refresh-users-btn', n_clicks=0,
                       style={'background-color': '#f39c12', 'color': 'white', 'border': 'none',
                              'padding': '8px 15px', 'border-radius': '5px', 'cursor': 'pointer', 'margin': '5px'}),
            html.Div(id='user-management-status', style={'marginTop': '10px'})
        ], style={'textAlign': 'center', 'padding': '15px', 'border': '1px solid #bdc3c7', 'border-radius': '5px', 'display': 'none'}),
        html.Div(id='users-table-container', style={'marginTop': '20px'})
    ], id='user-management-section', style={'display': 'none'}),
    
    # Analytics dashboard section
    html.Div([
        html.H4("Rescue Analytics Dashboard", style={'color': '#2c3e50', 'textAlign': 'center'}),
        
        # Key Metrics Cards
        html.Div([
            html.Div([
                html.H5("Water Rescue Eligible"),
                html.H3(id='water-rescue-count', children='0'),
                html.P("Intact Female Dogs"),
                html.Div(id='water-breakdown', style={'fontSize': '12px', 'marginTop': '5px'})
            ], style={'flex': 1, 'textAlign': 'center', 'padding': '10px', 'border': '1px solid #3498db', 'margin': '5px', 'borderRadius': '5px'}),
            
            html.Div([
                html.H5("Wilderness Rescue Eligible"),
                html.H3(id='wilderness-rescue-count', children='0'),
                html.P("Intact Male Dogs"),
                html.Div(id='wilderness-breakdown', style={'fontSize': '12px', 'marginTop': '5px'})
            ], style={'flex': 1, 'textAlign': 'center', 'padding': '10px', 'border': '1px solid #e74c3c', 'margin': '5px', 'borderRadius': '5px'}),
            
            html.Div([
                html.H5("Disaster Rescue Eligible"),
                html.H3(id='disaster-rescue-count', children='0'),
                html.P("Intact Male Dogs"),
                html.Div(id='disaster-breakdown', style={'fontSize': '12px', 'marginTop': '5px'})
            ], style={'flex': 1, 'textAlign': 'center', 'padding': '10px', 'border': '1px solid #2ecc71', 'margin': '5px', 'borderRadius': '5px'})
        ], style={'display': 'flex', 'justifyContent': 'space-around', 'margin': '10px 0'}),
        
        # Advanced Analytics Charts
        html.Div([
            html.Div([
                dcc.Graph(id='breed-performance-chart')
            ], style={'width': '48%', 'display': 'inline-block'}),
            html.Div([
                dcc.Graph(id='adoption-trends-chart')
            ], style={'width': '48%', 'display': 'inline-block', 'float': 'right'})
        ]),
        
        html.Div([
            dcc.Graph(id='demographics-chart')
        ])
        
    ], id='analytics-section', style={'display': 'none'}),
    
    # Main Dashboard Content
    html.Div(id='main-dashboard-content', style={'display': 'none'}, children=[
        html.Hr(),
        html.Center(img_html),
        html.Hr(),
        
        # Data table
        dash_table.DataTable(
            id='datatable-id',
            columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
            data=df.to_dict('records'),
            filter_action="native",
            sort_action="native",
            sort_mode="multi",
            page_action="native",
            page_current=0,
            page_size=10,
            style_table={'overflowX': 'scroll'},
            style_cell={
                'textAlign': 'left',
                'minWidth': '100px', 'width': '100px', 'maxWidth': '180px',
                'overflow': 'hidden',
                'textOverflow': 'ellipsis',
            }
        ),
        
        html.Hr(),
        
        # Interactive filtering
        html.Div([
            html.H5("Search and Rescue Filtering"),
            dcc.RadioItems(
                id='filter_type',
                options=[
                    {'label': 'Water Rescue', 'value': 'water-btn'},
                    {'label': 'Mountain or Wilderness Rescue', 'value': 'wilderness-btn'}, 
                    {'label': 'Disaster or Individual Tracking', 'value': 'disaster-btn'},
                    {'label': 'Reset', 'value': 'reset'}
                ], 
                value='reset', 
                inline=True,
                style={'padding': '10px'}
            )
        ], style={'textAlign': 'center'}),
        html.Hr(),
        
        # Map
        html.Div([
            dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
                dl.TileLayer(id="tile-layer"),
                dl.LayerGroup(id="marker-layer")
            ])
        ], style={'textAlign': 'center'})
    ]),
    
    html.Hr(),
    html.Footer("Ericka Resendez Dashboard with MongoDB Aggregation & JWT Authentication ")
])

#############################################
# Callbacks
#############################################

# Authentication callback
@app.callback(
    [Output('auth-status', 'children'),
     Output('user-info', 'children'),
     Output('login-btn', 'style'),
     Output('logout-btn', 'style'),
     Output('analytics-section', 'style'),
     Output('main-dashboard-content', 'style'),
     Output('user-management-section', 'style')],
    [Input('login-btn', 'n_clicks'),
     Input('logout-btn', 'n_clicks')],
    [State('username-input', 'value'),
     State('password-input', 'value')]
)
def handle_authentication(login_clicks, logout_clicks, username, password):
    global current_user, auth_token, user_role
    
    ctx = callback_context
    if not ctx.triggered:
        return "", "", {'display': 'inline-block'}, {'display': 'none'}, {'display': 'none'}, {'display': 'none'}, {'display': 'none'}
    
    button_id = ctx.triggered[0]['prop_id'].split('.')[0]
    
    if button_id == 'login-btn' and login_clicks > 0:
        if username and password:
            try:
                # Check if db connection exists
                if app.db is None:
                    return "Database connection not available - using demo mode", "", {'display': 'inline-block'}, {'display': 'none'}, {'display': 'none'}, {'display': 'none'}, {'display': 'none'}
                
                # Access db from app instead of global scope
                success, result = app.db.authenticate_user(username, password)
                if success:
                    current_user = username
                    auth_token = result
                    success, user_data = app.db.verify_token(auth_token)
                    if success:
                        user_role = user_data['role']
                        
                        # Show appropriate sections based on role
                        user_mgmt_style = {'display': 'block'} if user_role == 'admin' else {'display': 'none'}
                        
                        return (
                            f"Login successful! Welcome {username}",
                            f"Role: {user_role.upper()}",
                            {'display': 'none'},
                            {'display': 'inline-block'},
                            {'display': 'block'},
                            {'display': 'block'},
                            user_mgmt_style
                        )
                else:
                    return f"Login failed: {result}", "", {'display': 'inline-block'}, {'display': 'none'}, {'display': 'none'}, {'display': 'none'}, {'display': 'none'}
            except Exception as e:
                return f"Login error: {str(e)}", "", {'display': 'inline-block'}, {'display': 'none'}, {'display': 'none'}, {'display': 'none'}, {'display': 'none'}
        else:
            return "Please enter username and password", "", {'display': 'inline-block'}, {'display': 'none'}, {'display': 'none'}, {'display': 'none'}, {'display': 'none'}
    
    elif button_id == 'logout-btn' and logout_clicks > 0:
        try:
            if app.db is not None:
                app.db._log_audit_event(current_user, 'LOGOUT', "User logged out")
        except:
            pass  # Ignore logout audit errors
        current_user = None
        auth_token = None
        user_role = None
        return "Logged out successfully", "", {'display': 'inline-block'}, {'display': 'none'}, {'display': 'none'}, {'display': 'none'}, {'display': 'none'}
    
    return "", "", {'display': 'inline-block'}, {'display': 'none'}, {'display': 'none'}, {'display': 'none'}, {'display': 'none'}

# User Management Callbacks
@app.callback(
    [Output('user-management-status', 'children'),
     Output('users-table-container', 'children')],
    [Input('create-user-btn', 'n_clicks'),
     Input('refresh-users-btn', 'n_clicks')],
    [State('new-username', 'value'),
     State('new-password', 'value'),
     State('new-email', 'value'),
     State('new-role', 'value')]
)
def handle_user_management(create_clicks, refresh_clicks, new_username, new_password, new_email, new_role):
    global current_user, user_role
    
    ctx = callback_context
    if not ctx.triggered:
        return "", ""
    
    button_id = ctx.triggered[0]['prop_id'].split('.')[0]
    
    if not current_user or user_role != 'admin':
        return "Admin privileges required", ""
    
    if button_id == 'create-user-btn' and create_clicks > 0:
        if not all([new_username, new_password, new_role]):
            return "Please fill all required fields", ""
        
        try:
            if app.db is None:
                return "Database connection not available", ""
                
            success, message = app.db.create_user(current_user, new_username, new_password, new_role, new_email or '')
            if success:
                return f"Success: {message}", ""
            else:
                return f"Error: {message}", ""
        except Exception as e:
            return f"Error creating user: {str(e)}", ""
    
    elif button_id == 'refresh-users-btn' and refresh_clicks > 0:
        try:
            if app.db is None:
                return "Database connection not available", ""
                
            success, users = app.db.list_users(current_user)
            if success:
                # Create a table of users
                if users:
                    df_users = pd.DataFrame(users)
                    # Convert ObjectId to string for display
                    if '_id' in df_users.columns:
                        df_users['_id'] = df_users['_id'].astype(str)
                    
                    table = dash_table.DataTable(
                        id='users-table',
                        columns=[{"name": i, "id": i} for i in df_users.columns],
                        data=df_users.to_dict('records'),
                        style_cell={'textAlign': 'left'},
                        style_header={'backgroundColor': '#2c3e50', 'color': 'white', 'fontWeight': 'bold'}
                    )
                    return "Users list refreshed", table
                else:
                    return "No users found", ""
            else:
                return f"Error: {users}", ""
        except Exception as e:
            return f"Error loading users: {str(e)}", ""
    
    return "", ""

# Analytics Callbacks with error handling
@app.callback(
    [Output('water-rescue-count', 'children'),
     Output('wilderness-rescue-count', 'children'),
     Output('disaster-rescue-count', 'children'),
     Output('water-breakdown', 'children'),
     Output('wilderness-breakdown', 'children'),
     Output('disaster-breakdown', 'children')],
    [Input('analytics-section', 'style')]
)
def update_rescue_metrics(style):
    if style and style.get('display') == 'block':
        try:
            if app.db is None:
                return 25, 18, 32, "Labrador: 10 | Chesapeake: 8 | Newfoundland: 7", "German Shepherd: 6 | Malamute: 5 | Husky: 4 | Rottweiler: 3", "Doberman: 8 | German Shepherd: 7 | Golden Retriever: 9 | Bloodhound: 5 | Rottweiler: 3"
            
            rescue_data = app.db.get_rescue_type_analytics()
            
            water_count = rescue_data['water_rescue']['total']
            wilderness_count = rescue_data['wilderness_rescue']['total']
            disaster_count = rescue_data['disaster_rescue']['total']
            
            # Create breakdown text
            water_breakdown = " | ".join([f"{item['_id']}: {item['count']}" for item in rescue_data['water_rescue']['breakdown']])
            wilderness_breakdown = " | ".join([f"{item['_id']}: {item['count']}" for item in rescue_data['wilderness_rescue']['breakdown']])
            disaster_breakdown = " | ".join([f"{item['_id']}: {item['count']}" for item in rescue_data['disaster_rescue']['breakdown']])
            
            return water_count, wilderness_count, disaster_count, water_breakdown, wilderness_breakdown, disaster_breakdown
        except Exception as e:
            print(f"Error in analytics: {e}")
            return 25, 18, 32, "Demo data", "Demo data", "Demo data"
    
    return 0, 0, 0, "", "", ""

# Breed Performance Chart with error handling
@app.callback(
    Output('breed-performance-chart', 'figure'),
    [Input('analytics-section', 'style')]
)
def update_breed_performance_chart(style):
    if style and style.get('display') == 'block':
        try:
            if app.db is None:
                # Return demo data
                demo_data = [
                    {'breed': 'Labrador Retriever', 'adoption_rate': 85.2, 'success_rate': 92.1},
                    {'breed': 'German Shepherd', 'adoption_rate': 78.5, 'success_rate': 88.3},
                    {'breed': 'Golden Retriever', 'adoption_rate': 82.7, 'success_rate': 90.5},
                    {'breed': 'Beagle', 'adoption_rate': 75.9, 'success_rate': 84.2},
                    {'breed': 'Bulldog', 'adoption_rate': 70.3, 'success_rate': 79.8}
                ]
                df_breed = pd.DataFrame(demo_data)
            else:
                breed_data = app.db.get_breed_performance_metrics()
                df_breed = pd.DataFrame(breed_data)
            
            fig = go.Figure(data=[
                go.Bar(name='Adoption Rate %', x=df_breed['breed'], y=df_breed['adoption_rate'],
                      marker_color='#3498db'),
                go.Bar(name='Success Rate %', x=df_breed['breed'], y=df_breed['success_rate'],
                      marker_color='#2ecc71')
            ])
            
            fig.update_layout(
                title='Breed Performance Metrics (Adoption & Success Rates)',
                xaxis_title='Breed',
                yaxis_title='Rate (%)',
                barmode='group',
                height=400
            )
            return fig
        except Exception as e:
            print(f"Error in breed chart: {e}")
    
    # Return empty figure if no data
    return go.Figure()

# Adoption Trends Chart
@app.callback(
    Output('adoption-trends-chart', 'figure'),
    [Input('analytics-section', 'style')]
)
def update_adoption_trends_chart(style):
    if style and style.get('display') == 'block':
        try:
            if app.db is None:
                # Return demo data
                demo_data = [
                    {'month': 1, 'year': 2024, 'adoption_count': 45, 'dog_adoptions': 28, 'cat_adoptions': 15},
                    {'month': 2, 'year': 2024, 'adoption_count': 52, 'dog_adoptions': 32, 'cat_adoptions': 18},
                    {'month': 3, 'year': 2024, 'adoption_count': 48, 'dog_adoptions': 30, 'cat_adoptions': 16},
                    {'month': 4, 'year': 2024, 'adoption_count': 55, 'dog_adoptions': 35, 'cat_adoptions': 18},
                    {'month': 5, 'year': 2024, 'adoption_count': 60, 'dog_adoptions': 38, 'cat_adoptions': 20}
                ]
                df_trends = pd.DataFrame(demo_data)
            else:
                trends_data = app.db.get_monthly_adoption_trends()
                df_trends = pd.DataFrame(trends_data)
            
            df_trends['month_year'] = df_trends['month'].astype(str) + '/' + df_trends['year'].astype(str)
            
            fig = go.Figure()
            fig.add_trace(go.Scatter(x=df_trends['month_year'], y=df_trends['adoption_count'],
                                    mode='lines+markers', name='Total Adoptions', line=dict(color='#e74c3c')))
            fig.add_trace(go.Scatter(x=df_trends['month_year'], y=df_trends['dog_adoptions'],
                                    mode='lines+markers', name='Dog Adoptions', line=dict(color='#3498db')))
            fig.add_trace(go.Scatter(x=df_trends['month_year'], y=df_trends['cat_adoptions'],
                                    mode='lines+markers', name='Cat Adoptions', line=dict(color='#2ecc71')))
            
            fig.update_layout(
                title='Monthly Adoption Trends',
                xaxis_title='Month/Year',
                yaxis_title='Number of Adoptions',
                height=400
            )
            return fig
        except Exception as e:
            print(f"Error in trends chart: {e}")
    
    return go.Figure()

# Demographics Chart
@app.callback(
    Output('demographics-chart', 'figure'),
    [Input('analytics-section', 'style')]
)
def update_demographics_chart(style):
    if style and style.get('display') == 'block':
        try:
            if app.db is None:
                # Return demo data
                demo_data = [
                    {'animal_type': 'Dog', 'total_count': 65},
                    {'animal_type': 'Cat', 'total_count': 35},
                    {'animal_type': 'Bird', 'total_count': 12},
                    {'animal_type': 'Other', 'total_count': 8}
                ]
                df_demo = pd.DataFrame(demo_data)
            else:
                demo_data = app.db.get_animal_demographics()
                df_demo = pd.DataFrame(demo_data)
            
            fig = px.pie(df_demo, values='total_count', names='animal_type', 
                        title='Animal Type Distribution',
                        color_discrete_sequence=px.colors.qualitative.Set3)
            
            fig.update_layout(height=500)
            return fig
        except Exception as e:
            print(f"Error in demographics chart: {e}")
    
    return go.Figure()

if __name__ == '__main__':
    print("\n" + "="*50)
    print("Austin Animal Shelter Dashboard Starting...")
    print("="*50)
    print("\nDefault login credentials:")
    print("Username: admin, Password: admin234 (Admin role)")
    print("Username: user, Password: user123 (Viewer role)") 
    print("Username: analyst, Password: analyst456 (Analyst role)")
    print("\nOpen your browser and go to: http://127.0.0.1:8050")
    print("="*50)
    
    # Automatically open the browser
    import webbrowser
    webbrowser.open("http://127.0.0.1:8050")
    
    app.run(debug=True, host='127.0.0.1', port=8050)

Initializing Animal Shelter Database Connection...
Attempting to connect to MongoDB at localhost:27017...
MongoDB connection successful (no authentication)
Created default user: admin
Created default user: user
Created default user: analyst
Audit indexes created successfully
Database connection successful!
Loading initial data...
Successfully loaded 0 records

Austin Animal Shelter Dashboard Starting...

Default login credentials:
Username: admin, Password: admin234 (Admin role)
Username: user, Password: user123 (Viewer role)
Username: analyst, Password: analyst456 (Analyst role)

Open your browser and go to: http://127.0.0.1:8050


Error in trends chart: 'month'
Error in breed chart: 'breed'
Error in demographics chart: Value of 'names' is not the name of a column in 'data_frame'. Expected one of [] but received: animal_type
Error in demographics chart: Value of 'names' is not the name of a column in 'data_frame'. Expected one of [] but received: animal_typeError in breed chart: 'breed'

Error in trends chart: 'month'
